In [ ]:
#!pip install lazypredict

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


data_scores = pd.read_csv("/content/drive/MyDrive/linking-writing-processes-to-writing-quality-local/train_scores.csv")
data_scores.head()

# training_data features
data_features = pd.read_csv("/content/drive/MyDrive/linking-writing-processes-to-writing-quality-local/train_logs.csv")
data_features.head()

# Merge the features and scores on 'id'
data = pd.merge(data_features, data_scores, on='id', how='left')


import pandas as pd

def aggregate_data(df, agg_methods):
    # Perform the aggregation using the provided methods
    data_aggregated = df.groupby('id').agg(agg_methods).reset_index()

    # Flatten the MultiIndex columns and add the aggregation methods to the column names
    if isinstance(data_aggregated.columns, pd.MultiIndex):
        # Use a conditional expression to leave 'score' as is
        data_aggregated.columns = [f"{col[0]}_{col[1]}" if col[0] != 'score' else col[0] for col in data_aggregated.columns.values]

    return data_aggregated

# Define the aggregation methods outside of the function
aggregation_methods = {
    'down_time': ['sum'],
    'up_time': ['sum'],
    'action_time': ['sum'],
    'cursor_position': ['max'],
    'word_count': ['max'],
    'score': ['mean'],  # This will be left as 'score' in the column name
}

# Assume 'data' is a DataFrame already loaded
# Call the function with the data and the aggregation methods
data_aggregated = aggregate_data(data, aggregation_methods)
print(data_aggregated)

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Assuming data_aggregated is your DataFrame with the features and target

# Convert scores to string categories
y = data_aggregated['score'].apply(lambda x: '{:.1f}'.format(x))

# Define features
X = data_aggregated.drop(['id_', 'score'], axis=1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




           id_  down_time_sum  up_time_sum  action_time_sum  \
0     001519c8     2168798234   2169095477           297243   
1     0022f953     1273271023   1273546414           275391   
2     0042269b     3426641982   3427063183           421201   
3     0059420b     1222211589   1222401185           189596   
4     0075873a     1805499474   1805813176           313702   
...        ...            ...          ...              ...   
2466  ffb8c745     3488386746   3488886416           499670   
2467  ffbef7e5     2192480040   2192694261           214221   
2468  ffccd6fd     3764472937   3764704517           231580   
2469  ffec5b38     1869073112   1869362551           289439   
2470  fff05981     3897099261   3897400496           301235   

      cursor_position_max  word_count_max  score  
0                    1539             256    3.5  
1                    1676             323    3.5  
2                    2291             404    6.0  
3                    1047             2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from lazypredict.Supervised import LazyClassifier

# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit models and retrieve performance metrics
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Print model performance
print("\n")
print(models)

 90%|████████▉ | 26/29 [00:16<00:01,  2.59it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 1976, number of used features: 5
[LightGBM] [Info] Start training from score -5.979392
[LightGBM] [Info] Start training from score -4.154843
[LightGBM] [Info] Start training from score -3.563478
[LightGBM] [Info] Start training from score -3.298370
[LightGBM] [Info] Start training from score -2.513656
[LightGBM] [Info] Start training from score -2.001581
[LightGBM] [Info] Start training from score -1.638187
[LightGBM] [Info] Start training from score -1.609944
[LightGBM] [Info] Start training from score -1.786712
[LightGBM] [Info] Start training from score -2.633003
[LightGBM] [Info] Start training from score -2.963857
[LightGBM] [Info] Start training from score -4.187632
[LightGBM] [Warning] No further splits with positive gain, best g

100%|██████████| 29/29 [00:18<00:00,  1.54it/s]



                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LGBMClassifier                     0.31               0.24    None      0.30   
ExtraTreesClassifier               0.30               0.22    None      0.29   
RandomForestClassifier             0.31               0.22    None      0.29   
BaggingClassifier                  0.28               0.21    None      0.27   
LabelSpreading                     0.30               0.21    None      0.29   
LabelPropagation                   0.30               0.20    None      0.28   
LogisticRegression                 0.32               0.20    None      0.28   
SVC                                0.35               0.19    None      0.30   
DecisionTreeClassifier             0.24               0.19    None      0.25   
ExtraTreeClassifier                0.23               0.18    None      0.23   
KNeighborsClassifier               0.2

In [ ]:
# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model
dt_classifier.fit(X_train, y_train)

# Predict on training data for the training report and accuracy
y_train_pred = dt_classifier.predict(X_train)
training_accuracy = accuracy_score(y_train, y_train_pred)
training_report = classification_report(y_train, y_train_pred)

# Predict on validation set for the validation report and accuracy
y_val_pred = dt_classifier.predict(X_val)
validation_accuracy = accuracy_score(y_val, y_val_pred)
validation_report = classification_report(y_val, y_val_pred)

# Print training and validation accuracies
print(f"Training Accuracy: {training_accuracy}")
print(f"Validation Accuracy: {validation_accuracy}")

# Print classification reports
print("\nTraining Classification Report:\n", training_report)
print("Validation Classification Report:\n", validation_report)

Training Accuracy: 1.0
Validation Accuracy: 0.24444444444444444

Training Classification Report:
               precision    recall  f1-score   support

         0.5       1.00      1.00      1.00         5
         1.0       1.00      1.00      1.00        31
         1.5       1.00      1.00      1.00        56
         2.0       1.00      1.00      1.00        73
         2.5       1.00      1.00      1.00       160
         3.0       1.00      1.00      1.00       267
         3.5       1.00      1.00      1.00       384
         4.0       1.00      1.00      1.00       395
         4.5       1.00      1.00      1.00       331
         5.0       1.00      1.00      1.00       142
         5.5       1.00      1.00      1.00       102
         6.0       1.00      1.00      1.00        30

    accuracy                           1.00      1976
   macro avg       1.00      1.00      1.00      1976
weighted avg       1.00      1.00      1.00      1976

Validation Classification Report:
 

In [ ]:
data_aggregated

,id_,down_time_sum,up_time_sum,action_time_sum,cursor_position_max,word_count_max,score
0,001519c8,2168798234,2169095477,297243,1539,256,3.50
1,0022f953,1273271023,1273546414,275391,1676,323,3.50
2,0042269b,3426641982,3427063183,421201,2291,404,6.00
3,0059420b,1222211589,1222401185,189596,1047,206,2.00
4,0075873a,1805499474,1805813176,313702,1402,252,4.00
...,...,...,...,...,...,...,...
2466,ffb8c745,3488386746,3488886416,499670,1634,461,3.50
2467,ffbef7e5,2192480040,2192694261,214221,1877,438,4.00
2468,ffccd6fd,3764472937,3764704517,231580,2761,201,1.50
2469,ffec5b38,1869073112,1869362551,289439,2133,413,5.00


In [ ]:
data_scores['score'].nunique()

12

In [ ]:
1/12

0.08333333333333333

In [ ]:
data_aggregated

,id_,down_time_sum,up_time_sum,action_time_sum,cursor_position_max,word_count_max,score
0,001519c8,2168798234,2169095477,297243,1539,256,3.50
1,0022f953,1273271023,1273546414,275391,1676,323,3.50
2,0042269b,3426641982,3427063183,421201,2291,404,6.00
3,0059420b,1222211589,1222401185,189596,1047,206,2.00
4,0075873a,1805499474,1805813176,313702,1402,252,4.00
...,...,...,...,...,...,...,...
2466,ffb8c745,3488386746,3488886416,499670,1634,461,3.50
2467,ffbef7e5,2192480040,2192694261,214221,1877,438,4.00
2468,ffccd6fd,3764472937,3764704517,231580,2761,201,1.50
2469,ffec5b38,1869073112,1869362551,289439,2133,413,5.00
